synchronize your pydoxtools directory:

- rclone sync pydoxtools/ xyntopia_gdrive:/pydoxtools -P --size-only --fast-list

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
PDX_DIR="/content/pydoxtools"

In [3]:
%env TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data

env: TRAINING_DATA_DIR=/content/gdrive/MyDrive/pydoxtools/training_data


In [4]:
!pip install faker 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!ls "/content/gdrive/My Drive/pydoxtools/pydoxtools"
!rm -r $PDX_DIR
!cp -r "/content/gdrive/My Drive/pydoxtools/pydoxtools" $PDX_DIR

classifier.py	    extract_tables.py	      ocr_language_mappings.py
class_utils.py	    extract_textstructure.py  pdf_utils.py
cluster_utils.py    file_utils.py	      __pycache__
document.py	    html_utils.py	      qamachine.py
extract_classes.py  initialize_library.py     random_data_generators.py
extract_files.py    __init__.py		      settings.py
extract_html.py     labeling.py		      training.py
extract_index.py    list_utils.py	      visual_document_analysis.py
extract_logic.py    math_utils.py	      webdav_utils.py
extract_objects.py  models.py
extract_spacy.py    nlp_utils.py


The main issue when using google colab for our training is, that we only have python version 3.9 available, while pydoxtools currently needs 3.10 to function.and

because of this we are trying to only use the relevant parts of the library which is mainly the data generation part which is used for
the training.

In [6]:
!rm $PDX_DIR/__init__.py
!touch $PDX_DIR/__init__.py
!pwd

/content


finally...  run pydoxtools

In [7]:
from pydoxtools import random_data_generators

## prepare training

In [8]:
bg = random_data_generators.TextBlockGenerator.std_generator()
bg.classmap, bg.classmap_inv, bg.num_generators, bg.class_gen, bg.gen_mapping, bg.weights

({0: 'address', 1: 'unknown'},
 {'address': 0, 'unknown': 1},
 3,
 (<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f0b54fe69d0>,
  <pydoxtools.random_data_generators.RandomListGenerator at 0x7f0abb0c0400>),
 {<pydoxtools.random_data_generators.BusinessAddressGenerator at 0x7f0b54fe69d0>: 'address',
  <pydoxtools.random_data_generators.RandomTextBlockGenerator at 0x7f0abb0c0160>: 'unknown',
  <pydoxtools.random_data_generators.RandomListGenerator at 0x7f0abb0c0400>: 'unknown'},
 (100, 80, 20))

In [9]:
# %%
bgi = bg.__iter__()

In [10]:
for p in [next(bgi) for i in range(100)]:
    print(f"{p[1]} {bg.classmap[p[1].item()]}:\n{p[0]}\n\n")

1 unknown:
),T.ISLOADED=!1,T.ISREVEALED=!1)}),N(E).APPENDTO(T.$SLIDE),N(E).IS("VIDEO,AUDIO")&&(N(E).ADDCLASS("FANCYBOX-VIDEO"),N(E).WRAP("
"),T.CONTENTTYPE="VIDEO",T.OPTS.WIDTH=T.OPTS.WIDTH||N(E).ATTR("WIDTH"),T.OPTS.HEIGHT=T.OPTS.HEIGHT||N(E).ATTR("HEIGHT")),T.$CONTENT=T.$SLIDE.CHILDREN().FILTER("DIV,FORM,MAIN,VIDEO,AUDIO,ARTICLE,.FANCYBOX-CONTENT").FIRST(),T.$CONTENT.SIBLINGS().HIDE(),T.$CONTENT.LENGTT||(T.$CONTENT=T.$SLIDE.WRAPINNER("
").CHILDREN().IRS


1 unknown:
sons including availability of p at our our
M order will be shipped in multiple shipments? What does this mean?
Don’t worry! This is a completely situation. This just means that different parts of your This may have simply been shipped morf from different sGIT Are there any shipping charges applicable on my order?
We have sons shipping shipping for most addresses.
India shipping charges - A flat Rs.70 charg


0 address:
6550 Rua Engenheiro Luís Afonso Gomes
 Рованці· Харківська область ; 33093


0 address:
5528 Le Le grig